In [ ]:
# !pip install arize-otel opentelemetry-api opentelemetry-sdk arize-toolkit arize-phoenix-otel
# !pip install opentelemetry-exporter-otlp-proto-grpc
# !pip install git+https://github.com/strands-agents/sdk-python.git

In [ ]:
import os

API_KEY = "ak-976fc8b6-a6f4-4d33-910b-57b7a5e82444-tR_uNt1bn1ycYy3DVY0feMad-HyBI4WX"
SPACE_ID = "U3BhY2U6MjEzMTI6WTJqRA=="
ENDPOINT = "otlp.arize.com:443"
os.environ["ARIZE_SPACE_ID"] = SPACE_ID
os.environ["ARIZE_API_KEY"] = API_KEY
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = ENDPOINT

In [ ]:
import grpc
from arize.otel import register
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor

from strands_to_openinference_mapping import StrandsToOpenInferenceProcessor

provider = register(
    space_id=SPACE_ID,
    api_key=API_KEY,
    project_name="strands-agent-integration2",
    set_global_tracer_provider=True,
)

provider.add_span_processor(StrandsToOpenInferenceProcessor(debug=True))

provider.add_span_processor(
    BatchSpanProcessor(
        OTLPSpanExporter(
            endpoint=ENDPOINT,
            headers={
                "authorization": f"Bearer {API_KEY}",
                "api_key": API_KEY,
                "arize-space-id": SPACE_ID,
                "arize-interface": "python",
                "user-agent": "arize-python",
            },
            compression=grpc.Compression.Gzip,
        )
    )
)

trace.set_tracer_provider(provider)

In [ ]:
# Your Strands Agent Code Here:
# build you strands agent from here: https://github.com/strands-agents/samples/tree/main